In [42]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

registry = pd.read_csv("/Users/chun/Galvanize/capstones/capstone1-time-spleeping-cancer-correlation/data/CI5-XI/registry.txt",sep = "\t",encoding='iso-8859-1',header=0)
registry.columns = ['REGISTRY','country_long']

In [43]:
def clean_country_region(country_col):
    '''
    remove extra spaces, asterisk and (2008-2012)
    '''
    str1 = country_col.strip().replace('*','')
    str2 = str1.split('(')[0].strip()
    return str2

def get_country_star(country_col):
    '''
    get the columns with asterisk
    '''
    idx = country_col.find('*')
    if idx == -1:
        return False
    else:
        return True
    
def get_country(country_col):
    '''
    get the country name from the string
    '''
    str1 = country_col.strip()
    str2 = str1.split(',')[0].replace('*','')
    str3 = str2.split('(')[0].strip()
    str4 = str3.split(':')[0]
    str5 = str4.split(';')[0]
    return str5

def set_national(country_col):
    '''
    returns true if the country string indicates it's national
    '''
    if (country_col.find(':') == -1 and country_col.find(';') == -1 and country_col.find(',') == -1):
        return 1
    else:
        return 0

def set_is_subset(national_exists, is_national):
    if (national_exists == 0 and is_national == 0):
        return 1
    else:
        return 0
    
def std_country_region(country_region_col):
    '''
    country_col is a string value, standardize format
    ARGS:
        country_col string to transform
    RETURN:
        ISO recognized country name
    '''
    dict = {
        'Australian Capital Territory' : 'Australia',
        'Greater Poland':'Poland',
        'Iran':'Iran',
        'Republic of Korea':'Sounth Korea',                  
        'South Australia':'Australia',
        'The Netherlands':'Netherlands',
        'UK':'United Kingdom',
        'USA':'United States',
        'Western Australia':'Australia'    
    }
    if (dict.get(country_region_col) == None):
        return country_region_col
    else:
        return dict.get(country_region_col)

def munge_registry(registry):
    registry['country_region'] = list(map(clean_country_region, registry['country_long']))
    registry['country_region'] = list(map(std_country_region, registry['country_region']))
    registry['star'] = list(map(get_country_star, registry['country_long']))
    registry['country_name'] = list(map(get_country, registry['country_region']))
    registry['is_national'] = list(map(set_national, registry['country_region']))
    '''
    Some countries have cancer data at the national level and a sub-region by registry. Others have
    regions that need to be summed.
    Detect national registry by format of country name not containing ;:, then exclude all non
    national subsets for that country
    '''
    reg_group2 = registry.groupby('country_name')['is_national'].sum().reset_index()
    registry2 = pd.merge(registry,reg_group2,on=['country_name'],suffixes=['','_exists'])
    registry2['is_subset'] = list(map(set_is_subset,registry2['is_national_exists'],registry2['is_national']))
    df_reduced = registry2[(registry2['is_national'] == 1) | (registry2['is_subset'] == 1)]
    df_reduced.set_index('REGISTRY')
    return df_reduced

In [44]:
df_reg = munge_registry(registry)

In [45]:
df_reg

,REGISTRY,country_long,country_region,star,country_name,is_national,is_national_exists,is_subset
0,101200399,...,"Algeria, Batna",True,Algeria,0,0,1
1,140400299,...,"Kenya, Nairobi",True,Kenya,0,0,1
2,169000099,...,Seychelles,False,Seychelles,1,1,0
3,171000199,*South ...,"South Africa, Eastern Cape",True,South Africa,0,0,1
4,180000299,*Uga...,"Uganda, Kyadondo County",True,Uganda,0,0,1
...,...,...,...,...,...,...,...,...
450,603600199,Australi...,Australia,False,Australia,1,4,0
456,603600499,...,Australia,False,Australia,1,4,0
459,603600799,...,Australia,False,Australia,1,4,0
460,625009399,...,New Caledonia,True,New Caledonia,1,1,0


In [51]:
df_reg.country_name.value_counts()

United States    167
Italy             36
China             36
India             16
France            16
                ... 
Costa Rica         1
Zimbabwe           1
Kuwait             1
Cyprus             1
Latvia             1
Name: country_name, Length: 66, dtype: int64

In [52]:
df_reg[df_reg['country_name'] == 'United States']

,REGISTRY,country_long,country_region,star,country_name,is_national,is_national_exists,is_subset
50,263000099,...,"USA, Puerto Rico",False,United States,0,0,1
51,384000010,...,USA: White,False,United States,0,0,1
52,384000030,...,USA: Black,False,United States,0,0,1
53,384008099,...,"USA, NPCR",False,United States,0,0,1
54,384008010,...,"USA, NPCR: White",False,United States,0,0,1
...,...,...,...,...,...,...,...,...
212,684009920,*...,"USA, Hawaii: Chinese",True,United States,0,0,1
213,684009945,*U...,"USA, Hawaii: Hawaiian",True,United States,0,0,1
214,684009923,*U...,"USA, Hawaii: Filipino",True,United States,0,0,1
215,684009899,*...,"USA, Pacific Islands",True,United States,0,0,1


In [50]:
df_reg['country_name'] = list(map(std_country_region,df_reg['country_name']))

In [54]:
df_reg.to_csv(r'/Users/chun/Galvanize/capstones/capstone1-time-spleeping-cancer-correlation/data/clean_registry.csv',index = False, header = True)